In [14]:
import mne
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Dropout)

import itertools

from IPython.utils import io

from ast import literal_eval

**Run our data ingester to bring our initial data in**

In [10]:
%run data_ingester.py

### In this notebook, we'll build L1 models from neural networks using input from higher-dimension CSP components to capture a bit more signal for our final NN

Similarly to the LDA models, we'll shoot for accuracy of about 75%. However, because shallow NN are not as good at interpreting the CSP components as LDA models, we should be able to use a higher number of CSP components, and capture a different signal to pass into our ensemble.

We'll use the MNE parameters from our LDA/CSP models as the basis for testing NN on CSP.

In [37]:
#Import final CSP model params
base_params = pd.read_csv('data/csp_models_not_overfit.csv')

#Reset several columns away from strings and as literal_evals so can be read
base_params['trial_combo'] = (base_params['trial_combo'].
                                apply(lambda x: literal_eval(x)))

#Use list comprehension b/c NaN can appear in list
base_params['flat'] = [None if pd.isna(x) else literal_eval(x)
                         for x in base_params['flat']]
base_params['reject'] = [None if pd.isna(x) else literal_eval(x)
                         for x in base_params['reject']]

base_params['projectors_to_apply']

0    slice(None, 1, None)
1    slice(None, 1, None)
2    slice(None, 1, None)
3       slice(1, 2, None)
4    slice(None, 1, None)
5    slice(None, 1, None)
6    slice(None, 1, None)
7    slice(None, 1, None)
8       slice(1, 2, None)
Name: projectors_to_apply, dtype: object

In [38]:
base_params['projectors_to_apply'] = [slice(None, 1, None),
                                      slice(None, 1, None),
                                      slice(None, 1, None),
                                      slice(1, 2, None),
                                      slice(None, 1, None),
                                      slice(None, 1, None),
                                      slice(None, 1, None),
                                      slice(None, 1, None),
                                      slice(1, 2, None)]

base_params['test_acc'] = None
base_params['train_acc'] = None

#Columns to drop that we want to overwrite with new test params
base_params.drop(['n_components', 'cov_est', 'log'], 
                 inplace=True,
                 axis=1)

#Replace NaNs with Nones so MNE reads them properly
base_params.replace(np.nan, None, inplace=True)

#Create our test frame of parameters we want to iterate over
n_components_options = [8, 12, 16]
cov_est_options = ['concat', 'epoch']
log_options = [True, False]

columns = ['n_components',
           'cov_est',
           'log']

#Create dataframe of our variable params
variable_params = pd.DataFrame(itertools.
                               product(n_components_options,
                                       cov_est_options,
                                       log_options), 
                           columns=columns)

#Get number of permutations of variable parameters
permutations = len(list(itertools.
                        product(n_components_options,
                                cov_est_options,
                                log_options)))

#Duplicate our temp frame to match the number of variable
#permutations to run each permutation for each subject
temp_columns = base_params.columns
base_params = pd.DataFrame(np.repeat(base_params.values, 
                                     permutations, 
                                     axis=0))
base_params.columns = temp_columns

#Concat variable params with itself 9 times times to get
#right shape to combine with all params for all subjects
nn_csp_df = pd.concat((variable_params, 
                              variable_params,
                              variable_params,
                              variable_params,
                              variable_params,
                              variable_params,
                              variable_params,
                              variable_params,
                              variable_params),
                              ignore_index=True)

#Join our two grids to assemble full ensemble test grid
nn_csp_df = base_params.join(nn_csp_df)

nn_csp_df.shape

(108, 20)

### Let's construct the function we'll use to iterate across our test dataframe and find the right parameters for our NN / CSP models

In [31]:
def NN_csp_grid_search(test_df, savefile):
    """A function to iterate across a test
    dataframe and fill in the resulting
    train and test scores using those parameters.
    
    The test dataframe must be constructed with a 
    subject column, i.e., one subject per row.
    
    Savefile is the filename to use in saving
    test dataframe to csv in data directory
    each iteration."""
    for row in range(test_df.shape[0]):
        #Load each sessions data into an MNE raw object
        raw_dict = {}
        for key, value in data_dict.items():
            if (test_df.subject[row] in key) and ('sesh_1' in key):
                raw_dict[key] = mne.io.RawArray(value.T, info, verbose=0)


        #Filter data with bandpass. Note raw.filter applies in place
        for key, value in raw_dict.items():
            value.filter(l_freq=test_df.l_freq_filter[row], 
                         h_freq=test_df.h_freq_filter[row], 
                         method='fir', phase='zero', verbose=0)

        #Create epoch object with our raw objects and events arrays
        channels_to_keep = [ch for ch in ch_names if 
                            ch not in test_df.channels_to_drop[row]]
        epoch_dict = {}
        for key, value in raw_dict.items():
            epoch_dict[key] = mne.Epochs(value, events=event_dict[key], 
                                        event_id=events_explained, 
                                        tmin=-3, tmax=test_df.tmax[row], 
                                        baseline=test_df.baseline_correction[row],
                                        preload=True,
                                        picks=channels_to_keep, verbose=0,
                                        detrend=test_df.detrend[row],
                                        reject=test_df.reject[row],
                                        flat=test_df.flat[row],
                                        reject_tmin=test_df.tmin[row],
                                        reject_tmax=test_df.tmax[row])

        #Skip creating projectors step to save compute time if not being
        #applied in this iteration
        if test_df.projectors_to_apply[row]:
            #Create dictionary of signal space projection vectors for each epoch
            proj_dict = {}
            for key, value in epoch_dict.items():
                proj_dict[key] = mne.compute_proj_epochs(value, 
                                                         n_eeg=2, 
                                                         verbose=0)
            #apply projectors
            for key, value in epoch_dict.items():
                value.add_proj(proj_dict[key][test_df.projectors_to_apply[row]], 
                               verbose=0)
                value.apply_proj(verbose=0)

        #Skip creating ICA components step to save compute time if not
        #being applied in this iteration
        if test_df.ica_to_exclude[row]:
            #create and fit ICA object to epochs
            for key, value in epoch_dict.items():
                ica = mne.preprocessing.ICA(n_components=5, method='picard', 
                                            max_iter='auto', verbose=0)
                ica.fit(value, verbose=0)
                #Apply the ICA
                ica.apply(value, exclude=test_df.ica_to_exclude[row],
                         verbose=0)

        #Resample the data at a new frequency, happens inplace
        for key, value in epoch_dict.items():
            value.resample(sfreq=test_df.selected_frequency[row])

        #Extract and standard scale data from all non-dropped epochs
        #Creates intermediate data dictionary
        int_data_dict = {}
        #Use robust sklearn scaler
        if test_df.scaler[row] == 'robust':
            mne_scaler = mne.decoding.Scaler(scalings='median')
            for key, value in epoch_dict.items():
                #with scalings=median implements sklearn robust scaler
                int_data_dict[key] = (mne_scaler.
                                      fit_transform(value.
                                                    get_data(tmin=test_df.tmin[row], 
                                                             tmax=test_df.tmax[row])))
        #No scaling option
        if test_df.scaler[row] is None:
            for key, value in epoch_dict.items():
                int_data_dict[key] = value.get_data(tmin=test_df.tmin[row], 
                                                      tmax=test_df.tmax[row])

        #Create updated dictionary of y values to reflect dropped epochs
        int_y_dict = {}
        for key, value in y_dict.items():
            if (test_df.subject[row] in key) and ('sesh_1' in key):
                temp_y_list = []
                for i, epoch in enumerate(epoch_dict[key].drop_log):
            #MNE drop log shows empty parens for epochs that were not dropped - 
            #these are the trials we are keeping in each iteration
                    if epoch == ():
                        temp_y_list.append(value[i])
                int_y_dict[key] = temp_y_list

        #Assemble final y dict with only trials in our current combo
        #In each combo, coding 1st trial type to 0, 2nd trial type to 1
        final_y_dict = {}
        for key, value in int_y_dict.items():
            temp_y_list = []
            for y in value:
                if y == test_df.trial_combo[row][0]:
                    temp_y_list.append(0)
                if y == test_df.trial_combo[row][1]:
                    temp_y_list.append(1)
            final_y_dict[key] = np.array(temp_y_list)

        #Assemble data dict with only trials in our current combo
        final_data_dict = {}
        for key, value in int_data_dict.items():
            index_list = []
            for i, y in enumerate(int_y_dict[key]):
                if (y == test_df.trial_combo[row][0] or 
                    y == test_df.trial_combo[row][1]):
                    index_list.append(i)
            final_data_dict[key] = value[index_list]

        #Create csp_dict of csp objects
        csp_dict = {}
        for key, value in epoch_dict.items():
            csp_dict[key] = mne.decoding.CSP(n_components=int(test_df.n_components[row]), 
                                                 cov_est=test_df.cov_est[row], 
                                                 log=bool(test_df.log[row]));

        #Suppress output from this noisy function with no verbose option
        with io.capture_output() as captured:
        #Fit csp objects to training data from session 1        
            for key, value in csp_dict.items():
                #Try except to deal with iterations where fails to converge
                try:
                    value.fit(X=final_data_dict[key], 
                          y=final_y_dict[key]);
                except:
                    csp_dict[key] = 'CSP failed to converge'

        #Use csp objects to transform and save resulting data
        csp_data_dict = {}
        for key, value in csp_dict.items():
            #If except to deal with iterations where CSP fails to converge
            if value == 'CSP failed to converge':
                csp_data_dict[key] = 'CSP failed to converge'
            else:
                csp_data_dict[key] = value.transform(final_data_dict[key])

        #Model against our data for each subject and save the resulting score
        for key, value in csp_data_dict.items():

            #Pass through CSP failure to ouput
            if csp_dict[key] == 'CSP failed to converge':
                test_df.at[row, 'test_acc'] = 'CSP failed to converge'
                test_df.at[row, 'train_acc'] = 'CSP failed to converge'
            #Else train test split our data
            else:
                (X_train, X_test, 
                 y_train, y_test) = train_test_split(value, 
                                                 final_y_dict[key], 
                                                 stratify=final_y_dict[key],
                                                 random_state=23)
            
                #Build model
                model = Sequential()
                #inputs qre equal to n_components created via CSP
                model.add(Dense(test_df.n_components[row], 
                                input_dim=test_df.n_components[row], 
                                activation='relu'))
                model.add(Dropout(0.2))
                #Add hidden layer with half as many nodes as input
                model.add(Dense(int(test_df.n_components[row]/2), 
                                activation='relu'))
                model.add(Dropout(0.2))
                #Hidden layer with 1/4 as many nodes as input
                model.add(Dense(int(test_df.n_components[row]/4), 
                                activation='relu'))
                model.add(Dropout(0.2))
                #output layer
                model.add(Dense(1, activation='sigmoid'))

                #Suppress output
                with io.capture_output() as captured:
                    #Compile model
                    model.compile(loss='binary_crossentropy', 
                                  optimizer='adam', 
                                  metrics=['acc'])

                    #Fit model
                    key2 = key.replace('1', '2')
                    history = model.fit(X_train, y_train, 
                                        validation_data=(X_test, 
                                                         y_test), 
                                        epochs=3, verbose=0)

                #Save validation accuracy into dataframe
                test_df.at[row, 'test_acc'] = max(history.history['val_acc'])
                test_df.at[row, 'train_acc'] = max(history.history['acc'])

        test_df.to_csv(f'data/{savefile}.csv', index=False)
        if row % 50 == 0:
            print(f'Grid search complete through row {row} of {test_df.shape[0]}')

### Let's run through our test frame and find models that match our 75% accuracy goal, and with train and test accuracies that are quite close

In [39]:
NN_csp_grid_search(test_df=nn_csp_df, savefile='NN_from_CSP_gridsearch_1')

Grid search complete through row 0 of 108
Grid search complete through row 50 of 108
Grid search complete through row 100 of 108


In [52]:
nn_csp_df[nn_csp_df['subject'] == 'sub_L'].sort_values(
    'train_acc', ascending=False)[:10]

,subject,trial_combo,flat,reject,baseline_correction,detrend,ica_to_exclude,scaler,test_acc,l_freq_filter,h_freq_filter,channels_to_drop,projectors_to_apply,selected_frequency,tmin,tmax,train_acc,n_components,cov_est,log
103,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.842105,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.709091,12,epoch,False
97,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.578947,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.636364,8,concat,False
104,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.473684,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.636364,16,concat,True
99,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.526316,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.6,8,epoch,False
101,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.526316,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.6,12,concat,False
100,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.789474,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.581818,12,concat,True
105,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.631579,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.581818,16,concat,False
106,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.473684,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.553571,16,epoch,True
102,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.526316,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.545455,12,epoch,True
96,sub_L,"(1, 4)",{'eeg': 29.5},{'eeg': 100},None,None,None,robust,0.473684,1.0,40.0,"['AFz', 'F7', 'F8', 'T3', 'T4', 'P7', 'PO3', '...","slice(1, 2, None)",256,1,5.5,0.509091,8,concat,True


**Based on the dataframe above, let's assemble our selections of final NN/CSP models**

Looked through the list of output manually to find models that were very similar on train and test accuracy and 75-80% accurate.

In [53]:
selected_index_list = [3, 16, 35, 41, 50, 65, 81, 88, 103]

In [55]:
nn_csp_df.iloc[selected_index_list].to_csv('data/NN_CSP_models_not_overfit.csv',
                                          index=False)